In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle

2023-01-31 11:24:22.901292: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


Load datasets and standardize data

In [2]:
def load_dataset(dsname):
    metadata = pd.read_csv(f"/datasets/nicolas_facchinetti/processed_data/{dsname}/metadata_final.csv")
    x = pickle.load(open(f"/datasets/nicolas_facchinetti/processed_data/{dsname}/processed_data0.p", "rb" ))
    y = pickle.load(open(f"/datasets/nicolas_facchinetti/processed_data/{dsname}/processed_labels.p", "rb" ))
    return metadata, x, y

In [3]:
from sklearn.preprocessing import StandardScaler

datasets = ["emodb", "emovo", "ravdess"]
scalers = {d: StandardScaler() for d in datasets}

data = {}
for d in datasets:
    md, x, y = load_dataset(d)
    data[d] = {}
    # standardize data
    n = x.shape
    data[d]["x"] = scalers[d].fit_transform(x.reshape(n[0],-1)).reshape(n)
    data[d]["y"] = y
    data[d]["metadata"] = md

In [4]:
data['emodb']['metadata'].head()

,path,filename,chunk,label,actor,gender,mod
0,/datasets/nicolas_facchinetti/emodb/data/13a05...,13a05Tc.wav,0,sad,13,f,0
1,/datasets/nicolas_facchinetti/emodb/data/13a05...,13a05Tc.wav,1,sad,13,f,0
2,/datasets/nicolas_facchinetti/emodb/data/13a05...,13a05Tc.wav,0,sad,13,f,1
3,/datasets/nicolas_facchinetti/emodb/data/13a05...,13a05Tc.wav,1,sad,13,f,1
4,/datasets/nicolas_facchinetti/emodb/data/13a05...,13a05Tc.wav,0,sad,13,f,1


Get train/test data

In [5]:
from sklearn.model_selection import train_test_split

train = {}
test = {}
test_md = {}
for d in datasets:
    train[d] = {}
    test[d] = {}
    train_df, test_df = train_test_split(data[d]['metadata'], 
                                       test_size = 0.2, 
                                       random_state = 1938,
                                       stratify = data[d]['metadata']['label'])
    test_md[d] = test_df.reset_index()
    train_index = train_df.index
    test_index = test_df.index
    print(f'For {d}:\ttrain size {len(train_index)}, test size {len(test_index)}')
    train[d]['x'] = data[d]['x'][train_index]
    train[d]['y'] = data[d]['y'][train_index]
    test[d]['x']= data[d]['x'][test_index]
    test[d]['y']= data[d]['y'][test_index]
    

For emodb:	train size 3344, test size 837
For emovo:	train size 3944, test size 987
For ravdess:	train size 7161, test size 1791


In [6]:
for d in datasets:
    print(f'For {d} the gender balance in test set is\n{test_md[d].gender.value_counts()}\n')

For emodb the gender balance in test set is
f    522
m    315
Name: gender, dtype: int64

For emovo the gender balance in test set is
f    499
m    488
Name: gender, dtype: int64

For ravdess the gender balance in test set is
f    896
m    895
Name: gender, dtype: int64



In [7]:
test_md['emodb'].head()

,index,path,filename,chunk,label,actor,gender,mod
0,2272,/datasets/nicolas_facchinetti/emodb/data/08b03...,08b03Tc.wav,2,sad,8,f,0
1,1115,/datasets/nicolas_facchinetti/emodb/data/14a07...,14a07Tc.wav,0,sad,14,f,2
2,4069,/datasets/nicolas_facchinetti/emodb/data/16b01...,16b01Tb.wav,1,sad,16,f,2
3,3325,/datasets/nicolas_facchinetti/emodb/data/16a07...,16a07Fb.wav,0,happy,16,f,2
4,3585,/datasets/nicolas_facchinetti/emodb/data/14a07...,14a07Fd.wav,0,happy,14,f,1


In [8]:
def get_m_f_data(md):
    m = md[md.gender=='m'].index
    f = md[md.gender=='f'].index
    return m, f

In [9]:
test_mf = {}
for d in datasets:
    data_d = get_m_f_data(test_md[d])
    print(f"For {d}\n\tmale:{data_d[0].shape[0]}\n\tfemale:{data_d[1].shape[0]}\n")

For emodb
	male:315
	female:522

For emovo
	male:488
	female:499

For ravdess
	male:895
	female:896



Load best parameter for each dataset

In [10]:
import keras_tuner as kt
param = pickle.load(open('/datasets/nicolas_facchinetti/param.p', "rb" ))
param  

{'emodb': {'dropout': 0.3,
  'lstm_dropout': 0.2,
  'learning_rate': 0.001,
  'batch_size': 32,
  'tuner/epochs': 120,
  'tuner/initial_epoch': 8,
  'tuner/bracket': 1,
  'tuner/round': 1,
  'tuner/trial_id': '0050',
  'lstm_units': 2},
 'emovo': {'dropout': 0.3,
  'lstm_dropout': 0.0,
  'learning_rate': 0.0001,
  'batch_size': 8,
  'tuner/epochs': 120,
  'tuner/initial_epoch': 8,
  'tuner/bracket': 1,
  'tuner/round': 1,
  'tuner/trial_id': '0001',
  'lstm_units': 2},
 'ravdess': {'dropout': 0.6,
  'lstm_dropout': 0.2,
  'learning_rate': 0.001,
  'batch_size': 128,
  'tuner/epochs': 120,
  'tuner/initial_epoch': 0,
  'tuner/bracket': 0,
  'tuner/round': 0,
  'lstm_units': 2}}

# Model definition

In [12]:
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout, MaxPooling2D, LSTM, TimeDistributed, InputLayer, Reshape, BatchNormalization, Bidirectional

In [13]:
def get_m(best_param):
    # Define hyper model architecture
    m = Sequential([
        InputLayer(input_shape=(261,128,1)),
        Reshape((9,29,128,1)),
        TimeDistributed(Conv2D(16, kernel_size=(5,5), activation='relu')),
        TimeDistributed(BatchNormalization()),
        TimeDistributed(MaxPooling2D(pool_size=(4,4), strides=2)),

        TimeDistributed(Conv2D(32, kernel_size=(3,3), activation='relu')),
        TimeDistributed(MaxPooling2D(pool_size=(2,2), strides=2)),

        TimeDistributed(Conv2D(64, kernel_size=(3,3), activation='relu')),
        TimeDistributed(MaxPooling2D(pool_size=(2,2), strides=1)),
        TimeDistributed(Flatten()),
        
        Dropout(best_param['dropout']),
        
        Bidirectional(LSTM(256, dropout=best_param['lstm_dropout'], return_sequences=False)),
        Dense(5, activation='softmax')
    ])
    
    lr = best_param['learning_rate']
    
    # Define optimizer, loss, and metrics
    m.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
              loss='categorical_crossentropy',
              metrics=["accuracy"])
    return m

Models training

In [14]:
models = {}
for d in param:
    m = get_m(param[d])

    earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min', restore_best_weights=True)
    reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=6, verbose=1, min_delta=1e-4, mode='min')
    
    print(f'Training on {d} dataset')
    history = m.fit(train[d]['x'], train[d]['y'],
                    epochs=100,
                    batch_size=param[d]["batch_size"],
                    validation_split=0.2,
                    callbacks=[earlyStopping, reduce_lr_loss], verbose=2)
    print("\n\n\n")
    
    models[d] = m

2023-01-31 11:24:32.910068: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-01-31 11:24:32.911113: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-01-31 11:24:33.020015: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:15:00.0 name: NVIDIA TITAN V computeCapability: 7.0
coreClock: 1.455GHz coreCount: 80 deviceMemorySize: 11.77GiB deviceMemoryBandwidth: 607.97GiB/s
2023-01-31 11:24:33.020062: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2023-01-31 11:24:33.022346: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2023-01-31 11:24:33.022409: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.10
2023-0

Training on emodb dataset


2023-01-31 11:24:34.237005: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2023-01-31 11:24:34.253302: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3301490000 Hz


Epoch 1/100


2023-01-31 11:24:36.032888: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2023-01-31 11:24:36.211459: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


84/84 - 5s - loss: 1.1708 - accuracy: 0.5275 - val_loss: 1.4491 - val_accuracy: 0.4425
Epoch 2/100
84/84 - 1s - loss: 0.6891 - accuracy: 0.7271 - val_loss: 1.0118 - val_accuracy: 0.6024
Epoch 3/100
84/84 - 1s - loss: 0.4921 - accuracy: 0.8138 - val_loss: 0.7424 - val_accuracy: 0.7309
Epoch 4/100
84/84 - 1s - loss: 0.3770 - accuracy: 0.8650 - val_loss: 0.5409 - val_accuracy: 0.7743
Epoch 5/100
84/84 - 1s - loss: 0.2697 - accuracy: 0.9058 - val_loss: 0.3492 - val_accuracy: 0.8729
Epoch 6/100
84/84 - 1s - loss: 0.1779 - accuracy: 0.9477 - val_loss: 0.3054 - val_accuracy: 0.8924
Epoch 7/100
84/84 - 1s - loss: 0.1351 - accuracy: 0.9622 - val_loss: 0.2732 - val_accuracy: 0.9028
Epoch 8/100
84/84 - 1s - loss: 0.0833 - accuracy: 0.9817 - val_loss: 0.2715 - val_accuracy: 0.8999
Epoch 9/100
84/84 - 1s - loss: 0.0687 - accuracy: 0.9854 - val_loss: 0.2786 - val_accuracy: 0.9028
Epoch 10/100
84/84 - 1s - loss: 0.0430 - accuracy: 0.9903 - val_loss: 0.2879 - val_accuracy: 0.8999
Epoch 11/100
84/84 - 

Epoch 23/100
45/45 - 2s - loss: 0.0726 - accuracy: 0.9784 - val_loss: 0.2911 - val_accuracy: 0.9114
Epoch 24/100
45/45 - 2s - loss: 0.0644 - accuracy: 0.9818 - val_loss: 0.2802 - val_accuracy: 0.9197
Epoch 25/100
45/45 - 2s - loss: 0.0567 - accuracy: 0.9838 - val_loss: 0.3051 - val_accuracy: 0.9100
Epoch 26/100
45/45 - 2s - loss: 0.0632 - accuracy: 0.9811 - val_loss: 0.2689 - val_accuracy: 0.9197
Epoch 27/100
45/45 - 2s - loss: 0.0547 - accuracy: 0.9848 - val_loss: 0.2741 - val_accuracy: 0.9170
Epoch 28/100
45/45 - 2s - loss: 0.0537 - accuracy: 0.9852 - val_loss: 0.2486 - val_accuracy: 0.9232
Epoch 29/100
45/45 - 2s - loss: 0.0475 - accuracy: 0.9864 - val_loss: 0.2958 - val_accuracy: 0.9170
Epoch 30/100
45/45 - 2s - loss: 0.0429 - accuracy: 0.9888 - val_loss: 0.2802 - val_accuracy: 0.9142
Epoch 31/100
45/45 - 2s - loss: 0.0455 - accuracy: 0.9869 - val_loss: 0.2807 - val_accuracy: 0.9107
Epoch 32/100
45/45 - 2s - loss: 0.0450 - accuracy: 0.9873 - val_loss: 0.2480 - val_accuracy: 0.9323


Model testing

In [15]:
for d in models:
    loss, accuracy = models[d].evaluate(test[d]['x'], test[d]['y'], batch_size=param[d]["batch_size"])
    print(f'For {d} test loss: {loss} accuracy: {accuracy}')

27/27 [==============================] - 0s 8ms/step - loss: 0.3170 - accuracy: 0.9092
For emodb test loss: 0.31697770953178406 accuracy: 0.9091995358467102
124/124 [==============================] - 1s 5ms/step - loss: 0.3912 - accuracy: 0.8723
For emovo test loss: 0.3912019729614258 accuracy: 0.8723404407501221
14/14 [==============================] - 0s 17ms/step - loss: 0.3132 - accuracy: 0.9107
For ravdess test loss: 0.3131626844406128 accuracy: 0.910664439201355


Make ART model

In [16]:
from art.estimators.classification import TensorFlowV2Classifier

/home/nicolas/.conda/envs/thesis/lib/python3.9/site-packages/art/estimators/certification/__init__.py:13: UserWarning: PyTorch not found. Not importing DeepZ functionality
  warnings.warn("PyTorch not found. Not importing DeepZ functionality")


In [17]:
models['emodb'].summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 9, 29, 128, 1)     0         
_________________________________________________________________
time_distributed (TimeDistri (None, 9, 25, 124, 16)    416       
_________________________________________________________________
time_distributed_1 (TimeDist (None, 9, 25, 124, 16)    64        
_________________________________________________________________
time_distributed_2 (TimeDist (None, 9, 11, 61, 16)     0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 9, 9, 59, 32)      4640      
_________________________________________________________________
time_distributed_4 (TimeDist (None, 9, 4, 29, 32)      0         
_________________________________________________________________
time_distributed_5 (TimeDist (None, 9, 2, 27, 64)      1

In [18]:
minv = np.array([data[d]['x'].min() for d in models])
minv = {d: v for v, d in zip(np.floor(minv), models)}

maxv = np.array([data[d]['x'].max() for d in models])
maxv = {d: v for v, d in zip(np.ceil(maxv), models)}

In [19]:
minv, maxv

({'emodb': -5.0, 'emovo': -3.0, 'ravdess': -2.0},
 {'emodb': 7.0, 'emovo': 7.0, 'ravdess': 8.0})

In [20]:
classifiers = {d: TensorFlowV2Classifier(models[d],
                                         nb_classes=5,
                                         input_shape=(261,128,1),
                                         loss_object=tf.keras.losses.SparseCategoricalCrossentropy(),
                                         clip_values=(minv[d], maxv[d]),
                                         channels_first=False)
               for d in models}

# Model attack

In [31]:
import time


def attack(model, method, d, md, bs):
    start = time.time()
    x_adv = method.generate(d['x'], verbose=True)
    elapsed = time.time() - start
    
    print(f'Attack get {elapsed}')
    
    mi, fi = get_m_f_data(md)
    
    def compute_attack(model, adv, x, y):
        loss, accuracy = model.evaluate(adv, y, batch_size=bs, verbose=0)
        perturbation = np.mean(np.abs((adv - x)))
        print(f'\t\tloss: {loss:.5f}, accuracy: {accuracy:.5f}, avg perturbation: {perturbation:.5f}\n')
        return loss, accuracy, perturbation
    
    print('\tWhole test set')
    l1, a1, p1 = compute_attack(model, x_adv, d['x'], d['y'])
    print('\tOnly male data')
    l2, a2, p2 = compute_attack(model, x_adv[mi], d['x'][mi], d['y'][mi])
    print('\tOnly female data')
    l3, a3, p3 = compute_attack(model, x_adv[fi], d['x'][fi], d['y'][fi])
    
    def make_dict(v):
        return {'All': v[0], 'Male': v[1], 'Female': v[2]}
    
    return {'Loss': make_dict([l1,l2,l3]), 'Accuracy': make_dict([a1,a2,a3]), 'Perturbation': make_dict([p1,p2,p3])}, elapsed

In [32]:
results = {}
times = {}

In [50]:
def df_attack(d_res):
    reform = {(dataset, param, metric): values
              for dataset, inner1 in d_res.items()
              for param, inner2 in inner1.items()
              for metric, values in inner2.items()
              if metric != 'Loss'}
    mux = pd.MultiIndex.from_tuples(reform.keys())
    df = pd.DataFrame(list(reform.values()), index=mux)
    return df

In [52]:
def df_time(d_res):
    reform = {(dataset, param): value
              for dataset, inner1 in d_res.items()
              for param, value in inner1.items()}
    mux = pd.MultiIndex.from_tuples(reform.keys())
    df = pd.DataFrame(list(reform.values()), index=mux, columns=['Time'])
    return df

# White Box attack

In [49]:
from art.attacks.evasion import FastGradientMethod

epss = [0.25, 0.5, 0.75, 1, 1.25]
results['fgsm'] = {}
times['fgsm'] = {}

for d in models:
    results['fgsm'][d] = {}
    times['fgsm'][d] = {}
    for e in epss:
        print(f'Attacking {d} with FGSM eps={e}')
        fgsm = FastGradientMethod(estimator=classifiers[d], eps=e)

        res, runtime = attack(models[d], fgsm, test[d], test_md[d], param[d]["batch_size"])
        results['fgsm'][d][e] = res
        times['fgsm'][d][e] = runtime
    print("_____________________________")

Attacking emodb with FGSM eps=0.25
Attack get 1.6676230430603027
	Whole test set
		loss: 6.30713, accuracy: 0.17085, avg perturbation: 0.21394

	Only male data
		loss: 7.19170, accuracy: 0.11429, avg perturbation: 0.21370

	Only female data
		loss: 5.77334, accuracy: 0.20498, avg perturbation: 0.21408

Attacking emodb with FGSM eps=0.5
Attack get 1.6095926761627197
	Whole test set
		loss: 6.75274, accuracy: 0.12784, avg perturbation: 0.42787

	Only male data
		loss: 7.42254, accuracy: 0.09524, avg perturbation: 0.42739

	Only female data
		loss: 6.34855, accuracy: 0.14751, avg perturbation: 0.42816

Attacking emodb with FGSM eps=0.75
Attack get 1.5768837928771973
	Whole test set
		loss: 6.44006, accuracy: 0.11470, avg perturbation: 0.64181

	Only male data
		loss: 6.88369, accuracy: 0.09206, avg perturbation: 0.64109

	Only female data
		loss: 6.17236, accuracy: 0.12835, avg perturbation: 0.64224

Attacking emodb with FGSM eps=1
Attack get 1.4817683696746826
	Whole test set
		loss: 5.9

In [53]:
df_attack(results['fgsm'])

All      Male    Female
emodb   0.25 Accuracy      0.170848  0.114286  0.204981
             Perturbation  0.213936  0.213697  0.214081
        0.50 Accuracy      0.127838  0.095238  0.147510
             Perturbation  0.427872  0.427394  0.428161
        0.75 Accuracy      0.114695  0.092063  0.128352
             Perturbation  0.641809  0.641091  0.642243
        1.00 Accuracy      0.112306  0.092063  0.124521
             Perturbation  0.855744  0.854788  0.856322
        1.25 Accuracy      0.108722  0.088889  0.120690
             Perturbation  1.069678  1.068483  1.070397
emovo   0.25 Accuracy      0.072948  0.061475  0.084168
             Perturbation  0.218117  0.218172  0.218063
        0.50 Accuracy      0.069909  0.061475  0.078156
             Perturbation  0.436233  0.436344  0.436126
        0.75 Accuracy      0.075988  0.067623  0.084168
             Perturbation  0.654351  0.654515  0.654188
        1.00 Accuracy      0.077001  0.067623  0.086172
             Perturbation  0.872439  0.872658  0.872231
        1.25 Accuracy      0.081054  0.075820  0.086172
             Perturbation  1.090331  1.090594  1.090074
ravdess 0.25 Accuracy      0.145729  0.127374  0.164062
             Perturbation  0.197921  0.191780  0.204056
        0.50 Accuracy      0.188163  0.181006  0.195312
             Perturbation  0.395706  0.383389  0.408010
        0.75 Accuracy      0.203797  0.204469  0.203125
             Perturbation  0.592467  0.573860  0.611054
        1.00 Accuracy      0.207147  0.202235  0.212054
             Perturbation  0.784306  0.758748  0.809839
        1.25 Accuracy      0.198213  0.187709  0.208705
             Perturbation  0.961089  0.926421  0.995723

In [47]:
df_time(times['fgsm'])

Time
emodb   0.33  1.551730
        0.66  1.549049
        1.00  1.551334
emovo   0.33  1.771480
        0.66  1.724020
        1.00  1.725117
ravdess 0.33  3.291875
        0.66  3.120408
        1.00  3.192088

In [54]:
from art.attacks.evasion import SaliencyMapMethod

# amount of pertubration
thetas = [0.33, 0.66, 1]
results['jsma'] = {}

for d in models:
    results['jsma'][d] = {}
    for t in thetas:
        print(f'Attacking {d} with JSMA theta={t}')
        jsma = SaliencyMapMethod(classifier=classifiers[d],
                                 theta=t, # +1 from paper but test in [-1, +1]
                                 gamma=0.1, #0.145 from paper
                                 verbose=True)

        res, rr = attack(models[d], jsma, test[d], test_md[d], param[d]["batch_size"])
        results['fgsm'][d][e] = res
    print("_____________________________")

Attacking emodb with JSMA theta=0.33


JSMA:   0%|          | 0/837 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [67]:
from art.attacks.evasion import DeepFool

# amount of pertubration
results['df'] = {}
epsilon = [1, 5, 20]
for d in models:
    results['df'][d] = {}
    for e in epsilon:
        print(f'Attacking {d} with DeepFool eps={e}')
        df = DeepFool(classifier=classifiers[d],
                      nb_grads=5,
                      max_iter=e,
                      verbose=True)

        res, rr = attack(models[d], df, test[d], test_md[d], param[d]["batch_size"])
        results['df'][d][e] = res
    print("_____________________________")

Attacking emodb with DeepFool eps=1


DeepFool:   0%|          | 0/837 [00:00<?, ?it/s]

Attack get 141.12766671180725
	Whole test set
		loss: 5.35159, accuracy: 0.11828, avg perturbation: 1.89471

	Only male data
		loss: 5.79693, accuracy: 0.09841, avg perturbation: 1.85706

	Only female data
		loss: 5.08284, accuracy: 0.13027, avg perturbation: 1.91743

Attacking emodb with DeepFool eps=5


DeepFool:   0%|          | 0/837 [00:00<?, ?it/s]

Attack get 144.76467370986938
	Whole test set
		loss: 5.51903, accuracy: 0.06093, avg perturbation: 1.89447

	Only male data
		loss: 5.90937, accuracy: 0.06984, avg perturbation: 1.85797

	Only female data
		loss: 5.28349, accuracy: 0.05556, avg perturbation: 1.91650

Attacking emodb with DeepFool eps=20


...

Attack get 167.4050314426422
	Whole test set
		loss: 5.83863, accuracy: 0.08612, avg perturbation: 1.10493

	Only male data
		loss: 6.04007, accuracy: 0.06762, avg perturbation: 1.19204

	Only female data
		loss: 5.64162, accuracy: 0.10421, avg perturbation: 1.01973

Attacking emovo with DeepFool eps=20


DeepFool:   0%|          | 0/987 [00:00<?, ?it/s]

Attack get 168.3308138847351
	Whole test set
		loss: 5.83863, accuracy: 0.08612, avg perturbation: 1.10493

	Only male data
		loss: 6.04007, accuracy: 0.06762, avg perturbation: 1.19204

	Only female data
		loss: 5.64162, accuracy: 0.10421, avg perturbation: 1.01973

_____________________________
Attacking ravdess with DeepFool eps=1


DeepFool:   0%|          | 0/1791 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [43]:
from art.attacks.evasion import BasicIterativeMethod

# amount of pertubration
results['bim'] = {}
eps = [0.1, 0.5, 0.8]
for d in models:
    results['bim'][d] = {}
    for e in eps:
        print(f'Attacking {d} with BIM eps={e}')
        df = BasicIterativeMethod(estimator=classifiers[d],
                                  eps=e,
                                  verbose=True)

        res = attack(models[d], df, test[d], test_md[d], param[d]["batch_size"])
        results['bim'][d][e] = res
    print("_____________________________")

Attacking emodb with BIM eps=0.1


PGD - Batches: 0it [00:00, ?it/s]

	Whole test set
		loss: 7.88331, accuracy: 0.11111, avg perturbation: 0.06569

	Only male data
		loss: 8.85329, accuracy: 0.08571, avg perturbation: 0.06608

	Only female data
		loss: 7.29798, accuracy: 0.12644, avg perturbation: 0.06546

Attacking emodb with BIM eps=0.5


PGD - Batches: 0it [00:00, ?it/s]

	Whole test set
		loss: 27.62653, accuracy: 0.06930, avg perturbation: 0.28498

	Only male data
		loss: 27.70691, accuracy: 0.07302, avg perturbation: 0.28350

	Only female data
		loss: 27.57803, accuracy: 0.06705, avg perturbation: 0.28587

Attacking emodb with BIM eps=0.8


...

	Whole test set
		loss: 17.21029, accuracy: 0.05472, avg perturbation: 0.06190

	Only male data
		loss: 17.99829, accuracy: 0.05251, avg perturbation: 0.06122

	Only female data
		loss: 16.42317, accuracy: 0.05692, avg perturbation: 0.06258

Attacking ravdess with BIM eps=0.5


PGD - Batches: 0it [00:00, ?it/s]

	Whole test set
		loss: 41.17162, accuracy: 0.05416, avg perturbation: 0.27429

	Only male data
		loss: 41.54029, accuracy: 0.05475, avg perturbation: 0.27351

	Only female data
		loss: 40.80336, accuracy: 0.05357, avg perturbation: 0.27508

Attacking ravdess with BIM eps=0.8


PGD - Batches: 0it [00:00, ?it/s]

	Whole test set
		loss: 43.99225, accuracy: 0.05416, avg perturbation: 0.40761

	Only male data
		loss: 44.41031, accuracy: 0.05475, avg perturbation: 0.40632

	Only female data
		loss: 43.57465, accuracy: 0.05357, avg perturbation: 0.40891

_____________________________


In [140]:
from art.attacks.evasion import CarliniLInfMethod

for d in models:
    print(f'Attacking {d} with C&W LInf')
    cw = CarliniLInfMethod(classifier=classifiers[d],
                              max_iter=10,
                              learning_rate=0.01,
                              initial_const=1e0,
                              largest_const=2e0,
                              verbose=True)

    attack(models[d], cw, test[d], test_md[d], param[d]["batch_size"])

Attacking emodb with C&W LInf


C&W L_inf:   0%|          | 0/837 [00:00<?, ?it/s]

	Whole test set
		loss: 2.51218 accuracy: 0.06930, avg perturbation: 0.05085

	Only male data
		loss: 2.73026 accuracy: 0.08254, avg perturbation: 0.04878

	Only female data
		loss: 2.38058 accuracy: 0.06130, avg perturbation: 0.05210

Attacking emovo with C&W LInf


C&W L_inf:   0%|          | 0/987 [00:00<?, ?it/s]

	Whole test set
		loss: 3.90252 accuracy: 0.07700, avg perturbation: 0.02697

	Only male data
		loss: 4.17128 accuracy: 0.07172, avg perturbation: 0.02658

	Only female data
		loss: 3.63968 accuracy: 0.08216, avg perturbation: 0.02736

Attacking ravdess with C&W LInf


C&W L_inf:   0%|          | 0/1791 [00:00<?, ?it/s]

	Whole test set
		loss: 3.25371 accuracy: 0.05583, avg perturbation: 0.02460

	Only male data
		loss: 3.39439 accuracy: 0.06257, avg perturbation: 0.02272

	Only female data
		loss: 3.11318 accuracy: 0.04911, avg perturbation: 0.02647



In [46]:
from art.attacks.evasion import UniversalPerturbation

# amount of pertubration
results['up'] = {}
eps = [5, 10, 20]

for d in models:
    results['up'][d] = {}
    for e in eps:
        print(f'Attacking {d} with Universal Perturbation eps={e}')
        df = UniversalPerturbation(classifier=classifiers[d],
                                   attacker='fgsm',
                                   eps=e,
                                   verbose=True)

        res = attack(models[d], df, test[d], test_md[d], param[d]["batch_size"])
        results['up'][d][e] = res
    print("_____________________________")

Attacking emodb with Universal Perturbation eps=5


Universal perturbation:   0%|          | 0/20 [00:00<?, ?it/s]

	Whole test set
		loss: 2.02437, accuracy: 0.15771, avg perturbation: 2.18741

	Only male data
		loss: 2.11292, accuracy: 0.09206, avg perturbation: 2.18741

	Only female data
		loss: 1.97094, accuracy: 0.19732, avg perturbation: 2.18741

Attacking emodb with Universal Perturbation eps=10


Universal perturbation:   0%|          | 0/20 [00:00<?, ?it/s]

	Whole test set
		loss: 2.19206, accuracy: 0.14815, avg perturbation: 2.48435

	Only male data
		loss: 2.31193, accuracy: 0.07937, avg perturbation: 2.48434

	Only female data
		loss: 2.11973, accuracy: 0.18966, avg perturbation: 2.48435

Attacking emodb with Universal Perturbation eps=20


...

	Whole test set
		loss: 1.80678, accuracy: 0.18816, avg perturbation: 2.30164

	Only male data
		loss: 1.81198, accuracy: 0.18771, avg perturbation: 2.30164

	Only female data
		loss: 1.80158, accuracy: 0.18862, avg perturbation: 2.30164

Attacking ravdess with Universal Perturbation eps=10


Universal perturbation:   0%|          | 0/20 [00:00<?, ?it/s]

	Whole test set
		loss: 1.69832, accuracy: 0.19375, avg perturbation: 3.50511

	Only male data
		loss: 1.70510, accuracy: 0.19553, avg perturbation: 3.50511

	Only female data
		loss: 1.69155, accuracy: 0.19196, avg perturbation: 3.50511

Attacking ravdess with Universal Perturbation eps=20


Universal perturbation:   0%|          | 0/20 [00:00<?, ?it/s]

	Whole test set
		loss: 1.83424, accuracy: 0.19710, avg perturbation: 5.35757

	Only male data
		loss: 1.85394, accuracy: 0.19106, avg perturbation: 5.35757

	Only female data
		loss: 1.81456, accuracy: 0.20312, avg perturbation: 5.35757

_____________________________


# Black Box attack

In [ ]:
from art.attacks.evasion import ZooAttack

# amount of pertubration
results['zoo'] = {}
eps = [5]

for d in models:
    results['zoo'][d] = {}
    for e in eps:
        print(f'Attacking {d} with Zoo eps={e}')
        df = ZooAttack(classifier=classifiers[d],
                       max_iter=10,
                       verbose=True)

        res = attack(models[d], df, test[d], test_md[d], param[d]["batch_size"])
        results['zoo'][d][e] = res
    print("_____________________________")

Attacking emodb with Zoo eps=5


ZOO:   0%|          | 0/837 [00:00<?, ?it/s]

In [82]:
from art.attacks.evasion import PixelAttack

# amount of pertubration
results['pa'] = {}
eps = [5]

for d in models:
    results['pa'][d] = {}
    for e in eps:
        print(f'Attacking {d} with PixelAttack eps={e}')
        df = PixelAttack(classifier=classifiers[d],
                         verbose=True)

        res = attack(models[d], df, test[d], test_md[d], param[d]["batch_size"])
        results['pa'][d][e] = res
    print("_____________________________")


Attacking emodb with PixelAttack eps=5


Pixel threshold: 0it [00:00, ?it/s]

KeyboardInterrupt: 

In [31]:
from art.attacks.evasion import BoundaryAttack

# amount of pertubration
results['ba'] = {}
eps = [5]

for d in models:
    results['ba'][d] = {}
    for e in eps:
        print(f'Attacking {d} with PixelAttack eps={e}')
        df = BoundaryAttack(estimator=classifiers[d],
                            max_iter=100,
                            targeted=False,
                            verbose=True)

        res = attack(models[d], df, test[d], test_md[d], param[d]["batch_size"])
        results['ba'][d][e] = res
    print("_____________________________")

Attacking emodb with PixelAttack eps=5


Boundary attack:   0%|          | 0/837 [00:00<?, ?it/s]

Boundary attack - iterations:   0%|          | 0/100 [00:00<?, ?it/s]

Boundary attack - iterations:   0%|          | 0/100 [00:00<?, ?it/s]

Boundary attack - iterations:   0%|          | 0/100 [00:00<?, ?it/s]

...

Boundary attack - iterations:   0%|          | 0/100 [00:00<?, ?it/s]

Boundary attack - iterations:   0%|          | 0/100 [00:00<?, ?it/s]

Boundary attack - iterations:   0%|          | 0/100 [00:00<?, ?it/s]

Boundary attack - iterations:   0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 